# Banka z SQLAlchemy

Knjižnico `sqlalchemy` namestimo z ukazom
```bash
python -m pip install sqlalchemy
```
oziroma (če nimamo administratorskih pravic)
```bash
python -m pip install --user sqlalchemy
```

In [1]:
from sqlalchemy import Column, ForeignKey, Integer, String
from sqlalchemy.orm import relationship, declarative_base, sessionmaker
from sqlalchemy import create_engine

Definirajmo naše razrede (entitetne tipe).

In [2]:
Base = declarative_base()

class Kraj(Base):
    # Tako se bo imenavala tabela v bazi
    __tablename__ = 'kraj'
    
    # Atributi (lastnosti), ki sledijo se preslikajo v stolpce
    posta = Column(Integer, primary_key=True)
    kraj = Column(String(250))

class Oseba(Base):
    __tablename__ = 'oseba'
    
    id = Column(Integer, primary_key=True)               # Primarni ključ
    ime = Column(String(250), nullable=False)            # Ne sme biti NULL
    kraj_id = Column(Integer, ForeignKey('kraj.posta')) # tuji ključ
    # Lastnost, ki odraža povezavo do tabele kraj.
    # Ker je enolično določena, ne rabimo dodatnih parametrov.
    kraj = relationship(Kraj)

Generirajmo podatkovno bazo v SQLite (datoteko `banka-orm.db`) oziroma se z njo povežimo, če datoteka že obstaja.

Če želimo izpise stavkov SQL, nastavimo parameter `echo=True`. Tega v produkciji **ne** nastavimo.

In [3]:
engine = create_engine('sqlite:///banka-orm.db', echo=True)

Povežimo `engine` na razred `Base`.

In [4]:
Base.metadata.bind = engine

Tabele na bazi v tem trenutku še niso ustvarjene. Ustvarimo jih z naslednjim klicem.

In [5]:
Base.metadata.create_all(engine)

2025-06-05 10:24:27,533 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-06-05 10:24:27,534 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("kraj")
2025-06-05 10:24:27,535 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-06-05 10:24:27,536 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("kraj")
2025-06-05 10:24:27,537 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-06-05 10:24:27,537 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("oseba")
2025-06-05 10:24:27,538 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-06-05 10:24:27,538 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("oseba")
2025-06-05 10:24:27,539 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-06-05 10:24:27,540 INFO sqlalchemy.engine.Engine 
CREATE TABLE kraj (
	posta INTEGER NOT NULL, 
	kraj VARCHAR(250), 
	PRIMARY KEY (posta)
)


2025-06-05 10:24:27,540 INFO sqlalchemy.engine.Engine [no key 0.00036s] ()
2025-06-05 10:24:27,553 INFO sqlalchemy.engine.Engine 
CREATE TABLE oseba (
	id INTEGER NOT NULL,

Odprimo generator sej in ustvarimo novo sejo.

In [6]:
DBSessionMaker = sessionmaker(bind=engine)
session = DBSessionMaker()

Ustvarimo nov objekt razreda `Kraj` in ga dodamo v sejo.

In [7]:
nov_kraj = Kraj(posta=1000, kraj='Ljubljana')
session.add(nov_kraj)

Metoda `commit` shrani (spremenjene) objekte v seji v podatkovno bazo.

In [8]:
session.commit()

2025-06-05 10:26:17,095 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-06-05 10:26:17,096 INFO sqlalchemy.engine.Engine INSERT INTO kraj (posta, kraj) VALUES (?, ?)
2025-06-05 10:26:17,097 INFO sqlalchemy.engine.Engine [generated in 0.00077s] (1000, 'Ljubljana')
2025-06-05 10:26:17,098 INFO sqlalchemy.engine.Engine COMMIT


Ustvarimo nov objekt razreda `Oseba` in ga preko lastnosti `kraj` povežemo s prej ustvarjenim krajem.

In [9]:
nova_oseba = Oseba(ime='Janez', kraj=nov_kraj)
session.add(nova_oseba)
session.commit()

2025-06-05 10:27:44,851 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-06-05 10:27:44,854 INFO sqlalchemy.engine.Engine SELECT kraj.posta AS kraj_posta, kraj.kraj AS kraj_kraj 
FROM kraj 
WHERE kraj.posta = ?
2025-06-05 10:27:44,855 INFO sqlalchemy.engine.Engine [generated in 0.00063s] (1000,)
2025-06-05 10:27:44,857 INFO sqlalchemy.engine.Engine INSERT INTO oseba (ime, kraj_id) VALUES (?, ?)
2025-06-05 10:27:44,857 INFO sqlalchemy.engine.Engine [generated in 0.00047s] ('Janez', 1000)
2025-06-05 10:27:44,858 INFO sqlalchemy.engine.Engine COMMIT


Izvedimo poizvedbo `SELECT * FROM oseba`.

In [10]:
session.query(Oseba).all()

2025-06-05 10:29:22,518 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-06-05 10:29:22,520 INFO sqlalchemy.engine.Engine SELECT oseba.id AS oseba_id, oseba.ime AS oseba_ime, oseba.kraj_id AS oseba_kraj_id 
FROM oseba
2025-06-05 10:29:22,521 INFO sqlalchemy.engine.Engine [generated in 0.00096s] ()


Vrnimo prvi kraj in ga izpišimo.

In [11]:
kraj1 = session.query(Kraj).first()
print(kraj1)
print("KRAJ:", kraj1.kraj)

2025-06-05 10:29:41,154 INFO sqlalchemy.engine.Engine SELECT kraj.posta AS kraj_posta, kraj.kraj AS kraj_kraj 
FROM kraj
 LIMIT ? OFFSET ?
2025-06-05 10:29:41,155 INFO sqlalchemy.engine.Engine [generated in 0.00130s] (1, 0)
KRAJ: Ljubljana


Poiščimo vse osebe v kraju `kraj1` in jih izpišimo kot objekte.

In [12]:
session.query(Oseba).filter(Oseba.kraj == kraj1).all()

2025-06-05 10:30:53,518 INFO sqlalchemy.engine.Engine SELECT oseba.id AS oseba_id, oseba.ime AS oseba_ime, oseba.kraj_id AS oseba_kraj_id 
FROM oseba 
WHERE ? = oseba.kraj_id
2025-06-05 10:30:53,519 INFO sqlalchemy.engine.Engine [generated in 0.00107s] (1000,)


Poiščimo eno tako osebo ter jo vrnimo in izpišimo.

In [13]:
oseba1 = session.query(Oseba).filter(Oseba.kraj == kraj1).one()
print(oseba1)
print("IME:", oseba1.ime)
print("KRAJ:", oseba1.kraj)

2025-06-05 10:31:23,067 INFO sqlalchemy.engine.Engine SELECT oseba.id AS oseba_id, oseba.ime AS oseba_ime, oseba.kraj_id AS oseba_kraj_id 
FROM oseba 
WHERE ? = oseba.kraj_id
2025-06-05 10:31:23,068 INFO sqlalchemy.engine.Engine [cached since 29.55s ago] (1000,)
IME: Janez
KRAJ: <__main__.Kraj object at 0x78c872aa1100>
